# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [4]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [5]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  return [-3 * a**2 - (a**-0.5)/2, 3 * cos(3*b) + 2.5 * b**1.5, c**-2] # return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [6]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

h = 0.000001
a = 2
b = 3
c = 4

# -----------
numerical_grad = [(f(a+h, b, c) - f(a, b, c))/h , (f(a, b+h, c) - f(a, b, c))/h , (f(a, b, c+h) - f(a, b, c))/h] # return a numerical approx of [df/da, df/db, df/dc]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [7]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
numerical_grad2 = [(f(a+h, b, c) - f(a-h, b, c))/(2*h) , (f(a, b+h, c) - f(a, b-h, c))/(2*h) , (f(a, b, c+h) - f(a, b, c-h))/(2*h)] # return a better approx of [df/da, df/db, df/dc]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188


## section 2: support for softmax

In [8]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad

    out._backward = _backward

    return out

  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO

  def exp(self):
    out = Value(exp(self.data), (self,), 'exp')

    def _backward():
      self.grad += out.data * out.grad

    out._backward = _backward

    return out

  def log(self):
    out = Value(log(self.data), (self,), 'log')

    def _backward():
      self.grad += (1/self.data) * out.grad

    out._backward = _backward

    return out

# We implement a reverse addition that allows Python to make sense of expressions
# of the form `n + v` (which calls `n.__add__(v)`) where v is a Value object but `n` is not.
  def __radd__(self, other):
      return self + other

  def __mul__(self, other):
      other = other if isinstance(other, Value) else Value(other)
      out = Value(self.data * other.data, (self, other), '*')

      def _backward():
          self.grad += other.grad * out.grad
          other.grad += self.grad * out.grad

      out._backward = _backward

      return out

  def __pow__(self, other):
      assert isinstance(other, (int, float)) # support only integer or floating point powers
      out = Value(self.data**other, (self,), f'**{other}')

      def _backward():
          self.grad += other * (self.data**(other - 1)) * out.grad

      out._backward = _backward

      return out

  def __truediv__(self, other):
      out = self * other**-1

      def _backward():
          self.grad += (1 / other.data) * out.grad
          other.grad -= self.data * (other.data**-2) * out.grad

      out._backward = _backward

      return out

  def __neg__(self):
      out = self * -1

      def _backward():
          self.grad -= 1 * out.grad

      out._backward = _backward

      return out

# Note that since the methods `__truediv__(self, other)` and `__neg__(self)` are
# implemented in terms of the methods `__mul__(self, other)` and `__pow__(self, other)`
# which handle gradient propagation, the gradient computations in the formers should
# not be needed and could be removed to improve efficiency. But simply removing
# these gradient computations within `__truediv__(self, other)` and `__neg__(self)`
# makes that we're not computing the gradient of the loss function below correctly
# (we return `0.0`!). What should be done?

  # ------

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [9]:
# test of the `exp` function
x = Value(0.0)
y = x.exp()
print(y)

Value(data=1.0)


In [10]:
# test of the `log` function
x = Value(1.0)
y = x.log()
print(y)

Value(data=0.0)


In [11]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.04177257051535044
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625318
OK for dim 2: expected 0.005653302662216329, yours returns 0.0056533026622163285
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [12]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch

# Define the logits as a PyTorch tensor (of dim 1) with `requires_grad=True` to track gradients
logits = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad=True)

In [13]:
logits

tensor([ 0.,  3., -2.,  1.], requires_grad=True)

In [14]:
# Compute the softmax using torch's corresponding built-in function
probs = torch.softmax(logits, dim=0)

In [15]:
probs

tensor([0.0418, 0.8390, 0.0057, 0.1135], grad_fn=<SoftmaxBackward0>)

In [16]:
# Compute the negative log likelihood loss
loss = -torch.log(probs[3])

In [17]:
# Print the loss value
print(loss.item())

2.1755154132843018


In [18]:
# Perform backpropagation to compute gradients
loss.backward()

In [19]:
# gradient
logits.grad

tensor([ 0.0418,  0.8390,  0.0057, -0.8865])

In [20]:
# Check the gradients
for dim in range(4):
    ok = 'OK' if abs(logits.grad[dim].item() - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, PyTorch returns {logits.grad[dim].item()}")

OK for dim 0: expected 0.041772570515350445, PyTorch returns 0.041772566735744476
OK for dim 1: expected 0.8390245074625319, PyTorch returns 0.8390244841575623
OK for dim 2: expected 0.005653302662216329, PyTorch returns 0.005653302650898695
OK for dim 3: expected -0.8864503806400986, PyTorch returns -0.8864504098892212


In [21]:
# As you may expect, PyTorch has a built-in function for computing the negative
# log likelihood loss called `torch.nn.NLLLoss`. It expects the input to be log
# probabilities (which can be computed using `torch.nn.functional.log_softmax`)
# and a target label. Thus, below we verify again the gradient, but this time
# using the built-in log likelihood loss function of Pytorch.

import torch.nn.functional as F
# Logits
## `dtype=torch.float64` to use double-precision floating-point format  (optional)
logits = torch.tensor([0.0, 3.0, -2.0, 1.0], dtype=torch.float64, requires_grad=True)

# Use log_softmax instead of softmax to directly get log probabilities
log_probs = F.log_softmax(logits, dim=0)

# Target label
target = torch.tensor([3])

# Create the negative log likelihood loss function
nll_loss = torch.nn.NLLLoss()

# Compute the loss using NLLLoss
## `unsqueeze(0)` adds a dimension at the 0th position to the 1D tensor `log_probs`,
## a batch size dimension (here 1), making `log_probs` into a 2D tensor, i.e. a 1 X 4 matrix.
loss = nll_loss(log_probs.unsqueeze(0), target)

# Perform backpropagation
loss.backward()

# Print the loss value
print(loss.item())

# Check the gradients
for dim in range(4):
    ok = 'OK' if abs(logits.grad[dim].item() - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, PyTorch returns {logits.grad[dim].item()}")



2.1755153626167143
OK for dim 0: expected 0.041772570515350445, PyTorch returns 0.04177257051535046
OK for dim 1: expected 0.8390245074625319, PyTorch returns 0.839024507462532
OK for dim 2: expected 0.005653302662216329, PyTorch returns 0.005653302662216331
OK for dim 3: expected -0.8864503806400986, PyTorch returns -0.8864503806400987
